In [2]:
import numpy as np
import os
import sys
import multiprocessing
import time

# PyTorch dependencies
import torch as pt
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

# local dependencies
from sembps.bps import aptbps
from modelnet40 import load_modelnet40

MAIN_PATH = os.path.join(os.sep, 'media', 'add320', 'riccardo')
MOD40_PATH = os.path.join(MAIN_PATH, 'modelnet40')

LOGS_PATH = os.path.join(MOD40_PATH, 'logs2')
DATA_PATH = os.path.join(MOD40_PATH, 'data')

BPS_CACHE_FILE = os.path.join(DATA_PATH, 'bps_mlp_data.npz')
APTBPS_CACHE_FILE = os.path.join(DATA_PATH, 'aptbps_mlp_data.npz')

N_MODELNET_CLASSES = 40

N_BPS_POINTS = 512
BPS_RADIUS = 1.7

N_CPUS = multiprocessing.cpu_count()
N_GPUS = torch.cuda.device_count()

if N_GPUS > 0:
    DEVICE = 'cuda'
    print("GPU device found...")
else:
    DEVICE = 'cpu'
    print("GPU device not found, using %d CPU(s)..." % N_CPUS)

if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)


class ShapeClassifierMLP(nn.Module):

    def __init__(self, n_features, n_classes, hsize1=512,  hsize2=512, dropout1=0.8, dropout2=0.8):
        super(ShapeClassifierMLP, self).__init__()

        self.bn0 = nn.BatchNorm1d(n_features)
        self.fc1 = nn.Linear(in_features=n_features, out_features=hsize1)
        self.bn1 = nn.BatchNorm1d(hsize1)
        self.do1 = nn.Dropout(dropout1)
        self.fc2 = nn.Linear(in_features=hsize1, out_features=hsize2)
        self.bn2 = nn.BatchNorm1d(hsize2)
        self.do2 = nn.Dropout(dropout2)
        self.fc3 = nn.Linear(in_features=hsize2, out_features=n_classes)

    def forward(self, x):

        x = self.bn0(x)
        x = self.do1(self.bn1(F.relu(self.fc1(x))))
        x = self.do2(self.bn2(F.relu(self.fc2(x))))
        x = self.fc3(x)

        return x

def fit(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()


def test(model, device, test_loader, epoch_id):
    model.eval()
    test_loss = 0
    n_test_samples = len(test_loader.dataset)
    n_correct = 0
    with pt.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            n_correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= n_test_samples
    test_acc = 100.0 * n_correct / n_test_samples
    print(
        "Epoch {} test loss: {:.4f}, test accuracy: {}/{} ({:.2f}%)".format(epoch_id, test_loss, n_correct, n_test_samples, test_acc))

    return test_loss, test_acc


def prepare_data_loaders(n_parts=2):

    if n_parts == 1:
        APTBPS_CACHE_FILE = os.path.join(DATA_PATH, 'aptbps_mlp_data.npz')
    else:
        file_name = str(n_parts) + 'aptbps_mlp_data.npz'
        APTBPS_CACHE_FILE = os.path.join(DATA_PATH, file_name)
    
    if not os.path.exists(APTBPS_CACHE_FILE):

        # load modelnet point clouds
        xtr, ytr, xte, yte = load_modelnet40(root_data_dir=DATA_PATH)

        # this will normalise your point clouds and return scaler parameters for inverse operation
        xtr_normalized = aptbps.normalize(xtr)
        xte_normalized = aptbps.normalize(xte)

        # this will encode your normalised point clouds with random basis of 512 points,
        # each BPS cell containing l2-distance to closest point
        start = time.time()
        print("converting data to BPS representation..")
        print("number of basis points: %d" % N_BPS_POINTS)
        print("BPS sampling radius: %f" % BPS_RADIUS)
        print("converting train..")
        xtr_bps = aptbps.adaptive_encode(xtr_normalized, n_bps_points=N_BPS_POINTS, n_parts=n_parts, bps_cell_type='dists', radius=BPS_RADIUS) # train
        print("converting test..")
        xte_bps = aptbps.adaptive_encode(xte_normalized, n_bps_points=N_BPS_POINTS, n_parts=n_parts, bps_cell_type='dists', radius=BPS_RADIUS) # test
        end = time.time()
        total_training_time = (end - start) / 60
        print("conversion finished. ")
        print("saving cache file for future runs..")

        np.savez(APTBPS_CACHE_FILE, xtr=xtr_bps, ytr=ytr, xte=xte_bps, yte=yte)

    else:
        print("loading converted data from cache..")
        data = np.load(APTBPS_CACHE_FILE)
        xtr_bps = data['xtr']
        ytr = data['ytr']
        xte_bps = data['xte']
        yte = data['yte']


    # long() converts it to long
    # these datasets contain the data laready normalized and encoded with bps
    dataset_tr = pt.utils.data.TensorDataset(pt.Tensor(xtr_bps), pt.Tensor(ytr[:, 0]).long())

    train_loader = pt.utils.data.DataLoader(dataset_tr, batch_size=512, shuffle=True)

    dataset_te = pt.utils.data.TensorDataset(pt.Tensor(xte_bps), pt.Tensor(yte[:, 0]).long())
    test_loader = pt.utils.data.DataLoader(dataset_te, batch_size=512, shuffle=True)

    return train_loader, test_loader

GPU device found...


In [9]:
import csv

def main():
    
    for i in range(2, 3):

        train_loader, test_loader = prepare_data_loaders(i)
    
        n_bps_features = train_loader.dataset[0][0].shape[0]
    
        print("defining the model..")
        model = ShapeClassifierMLP(n_features=n_bps_features, n_classes=N_MODELNET_CLASSES)
    
        optimizer = pt.optim.Adam(model.parameters(), lr=1e-3)
    
        n_epochs = 1200
        pbar = range(0, n_epochs)
        test_accs = []
        test_losses = []
    
        print("training started..")
        model = model.to(DEVICE)
        
        # Train in parallel on multiple GPUs
        model = nn.DataParallel(model)
    
        start = time.time()
    
        for epoch_idx in pbar:
            fit(model, DEVICE, train_loader, optimizer)
            if epoch_idx == 1000:
                for param_group in optimizer.param_groups:
                    print("decreasing the learning rate to 1e-4..")
                    param_group['lr'] = 1e-4
            if epoch_idx % 10 == 0:
                test_loss, test_acc = test(model, DEVICE, test_loader, epoch_idx)
                test_accs.append(test_acc)
                test_losses.append(test_loss)
                if test_acc > 88.9:
                    file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
                    ckpt_path = os.path.join(LOGS_PATH, file_name)
                    pt.save(model.state_dict(), ckpt_path)
                    print("Model saved: %s" % ckpt_path)
            #if epoch_idx % 100 == 0:
            #    file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
            #    ckpt_path = os.path.join(LOGS_PATH, file_name)
            #    pt.save(model.state_dict(), ckpt_path)
            #    print("Model saved: %s" % ckpt_path)
    
        _, test_acc = test(model, DEVICE, test_loader, n_epochs)
    
        end = time.time()
        total_training_time = (end - start) / 60
    
        print("Training finished. Test accuracy: %f . Total training time: %f minutes." % (test_acc, total_training_time))
        file_name = str(i) + 'aptbps_mlp_model.h5'
        ckpt_path = os.path.join(LOGS_PATH, file_name)
    
        pt.save(model.state_dict(), ckpt_path)
    
        print("Model saved: %s" % ckpt_path)

    return


if __name__ == '__main__':
    main()


loading converted data from cache..
defining the model..
training started..


/home/riccardo/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


Epoch 0 test loss: -1.8871, test accuracy: 924/2468 (37.44%)


KeyboardInterrupt: 

In [18]:
import csv

def main():
    
    for i in range(3, 4):

        train_loader, test_loader = prepare_data_loaders(i)
    
        n_bps_features = train_loader.dataset[0][0].shape[0]
    
        print("defining the model..")
        model = ShapeClassifierMLP(n_features=n_bps_features, n_classes=N_MODELNET_CLASSES)
    
        optimizer = pt.optim.Adam(model.parameters(), lr=1e-3)
    
        n_epochs = 1200
        pbar = range(0, n_epochs)
        test_accs = []
        test_losses = []
    
        print("training started..")
        model = model.to(DEVICE)
        
        # Train in parallel on multiple GPUs
        model = nn.DataParallel(model)
    
        start = time.time()
    
        for epoch_idx in pbar:
            fit(model, DEVICE, train_loader, optimizer)
            if epoch_idx == 1000:
                for param_group in optimizer.param_groups:
                    print("decreasing the learning rate to 1e-4..")
                    param_group['lr'] = 1e-4
            test_loss, test_acc = test(model, DEVICE, test_loader, epoch_idx)
            test_accs.append(test_acc)
            test_losses.append(test_loss)
            if test_acc > 89.0:
                file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
                ckpt_path = os.path.join(LOGS_PATH, file_name)
                pt.save(model.state_dict(), ckpt_path)
                print("Model saved: %s" % ckpt_path)
            #if epoch_idx % 100 == 0:
            #    file_name = str(i) + 'aptbps_mlp_model' +'_epoch' + str(epoch_idx) + '.h5'
            #    ckpt_path = os.path.join(LOGS_PATH, file_name)
            #    pt.save(model.state_dict(), ckpt_path)
            #    print("Model saved: %s" % ckpt_path)
        
        accs_file_name = str(i) + 'accs_aptbps_mlp_model.csv'
        accs_file_path = os.path.join(LOGS_PATH, accs_file_name)
        
        with open(accs_file_path, 'w', newline='\n') as f:
            for item in test_accs:
                f.write("%s\n" % item)
                
        losses_file_name = str(i) + 'losses_aptbps_mlp_model.csv'
        losses_file_path = os.path.join(LOGS_PATH, losses_file_name)
        
        with open(losses_file_path, 'w', newline='\n') as ff:
            for item in test_losses:
                ff.write("%s\n" % item)
            
        _, test_acc = test(model, DEVICE, test_loader, n_epochs)
    
        end = time.time()
        total_training_time = (end - start) / 60
    
        print("Training finished. Test accuracy: %f . Total training time: %f minutes." % (test_acc, total_training_time))
        file_name = str(i) + 'aptbps_mlp_model.h5'
        ckpt_path = os.path.join(LOGS_PATH, file_name)
    
        pt.save(model.state_dict(), ckpt_path)
    
        print("Model saved: %s" % ckpt_path)

    return


if __name__ == '__main__':
    main()


loading ModelNet40 point clouds...
loaded 9840 training and 2468 test samples.
converting data to BPS representation..
number of basis points: 512
BPS sampling radius: 1.700000
converting train..
using 12 available CPUs for BPS encoding..


100%|██████████| 820/820 [04:15<00:00,  3.21it/s]


converting test..
using 12 available CPUs for BPS encoding..


100%|██████████| 206/206 [01:04<00:00,  3.19it/s]


conversion finished. 
saving cache file for future runs..
defining the model..
training started..
Epoch 0 test loss: -1.9006, test accuracy: 908/2468 (36.79%)
Epoch 1 test loss: -3.9717, test accuracy: 1404/2468 (56.89%)
Epoch 2 test loss: -5.1570, test accuracy: 1640/2468 (66.45%)
Epoch 3 test loss: -5.8222, test accuracy: 1756/2468 (71.15%)
Epoch 4 test loss: -6.2841, test accuracy: 1832/2468 (74.23%)
Epoch 5 test loss: -6.6374, test accuracy: 1876/2468 (76.01%)
Epoch 6 test loss: -7.0806, test accuracy: 1896/2468 (76.82%)
Epoch 7 test loss: -7.3370, test accuracy: 1936/2468 (78.44%)
Epoch 8 test loss: -7.4525, test accuracy: 1948/2468 (78.93%)
Epoch 9 test loss: -7.7997, test accuracy: 1973/2468 (79.94%)
Epoch 10 test loss: -8.0677, test accuracy: 1985/2468 (80.43%)
Epoch 11 test loss: -8.3072, test accuracy: 1993/2468 (80.75%)
Epoch 12 test loss: -8.4761, test accuracy: 1996/2468 (80.88%)
Epoch 13 test loss: -8.7180, test accuracy: 2001/2468 (81.08%)
Epoch 14 test loss: -8.7111, te

In [8]:

a
mylist = ['foo', 'bar']
with open(accs_file_path, 'w', newline='\n') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['epoch', 'acc'])
    csvwriter.writerow(mylist)